In [1]:
import geopandas as gpd
from dem_stitcher.geojson_io import read_geojson_gzip
import pandas as pd
import folium
import leafmap
from tqdm import tqdm

# Load the data

In [2]:
df_burst_subset = read_geojson_gzip('burst_extent_subset.geojson.zip')
df_burst_subset.head()

,geometry,jpl_burst_id,orbit_pass,count,is_val_burst,track_number
0,"POLYGON ((-97.90103 16.86358, -97.06559 17.014...",T005-008688-IW2,ASCENDING,66,False,5
1,"POLYGON ((-97.09889 17.06678, -96.35903 17.196...",T005-008688-IW3,ASCENDING,66,False,5
2,"POLYGON ((-98.69437 16.82087, -97.89903 16.966...",T005-008689-IW1,ASCENDING,66,False,5
3,"POLYGON ((-97.93485 17.03023, -97.09864 17.180...",T005-008689-IW2,ASCENDING,66,False,5
4,"POLYGON ((-97.13196 17.23328, -96.39139 17.362...",T005-008689-IW3,ASCENDING,66,False,5


In [3]:
df_rtc = pd.read_json('rtc_s1_table.json.zip')
df_rtc.head()

,rtc_s1_id,input_slc_id,jpl_burst_id,bursts_per_slc_input,rtc_s1_vv_url,rtc_s1_vh_url,rtc_s1_h5_url,acq_datetime
0,OPERA_L2_RTC-S1_T005-008688-IW2_20201010T00400...,S1A_IW_SLC__1SDV_20201010T004001_20201010T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-10 00:40:01
1,OPERA_L2_RTC-S1_T005-008688-IW2_20201022T00400...,S1A_IW_SLC__1SDV_20201022T004000_20201022T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-22 00:40:00
2,OPERA_L2_RTC-S1_T005-008688-IW2_20201103T00400...,S1A_IW_SLC__1SDV_20201103T004000_20201103T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-11-03 00:40:00
3,OPERA_L2_RTC-S1_T005-008688-IW2_20201115T00400...,S1A_IW_SLC__1SDV_20201115T004000_20201115T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-11-15 00:40:00
4,OPERA_L2_RTC-S1_T005-008688-IW2_20201127T00400...,S1A_IW_SLC__1SDV_20201127T004000_20201127T0040...,T005-008688-IW2,27,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-11-27 00:40:00


In [4]:
df_val_sites = gpd.read_file('../3_dist_sites/dist_hls_val_sites.geojson')
df_val_sites_subset = gpd.sjoin(df_val_sites, df_burst_subset[['jpl_burst_id', 'geometry']], predicate='intersects', how='inner').drop(columns=['index_right']).reset_index(drop=True)
df_val_sites_subset.head()

,site_id,change_label,change_time,last_observation_time,geometry,jpl_burst_id
0,4,VLmaj,2021-11-07,2021-11-02,POINT (19.33289 4.57490),T036-076228-IW3
1,13,VLmaj,2021-10-03,NaT,POINT (48.77698 31.75490),T108-230715-IW2
2,13,VLmaj,2021-10-03,NaT,POINT (48.77698 31.75490),T101-214979-IW1
3,22,VLmin,2021-12-19,2021-12-12,POINT (36.61178 35.52891),T014-028130-IW2
4,22,VLmin,2021-12-19,2021-12-12,POINT (36.61178 35.52891),T021-043823-IW2


In [5]:
df_val_sites_subset.to_file('val_sites_subset.geojson')

# Visualize Bursts and Select One

Sources:

1. https://leafmap.org/notebooks/13_geopandas/
2. For the style function: https://claude.ai/ chat

In [6]:
track_numbers = df_burst_subset.track_number.unique()
len(track_numbers)

58

In [7]:
import leafmap

# Add track numbers if they are blocking tracks with data you require
TRACKS_TO_EXCLUDE = [137, 70]

gdf = df_burst_subset.copy()
gdf.geometry = df_burst_subset.geometry.exterior

def f(feature):
    return {
        'fillColor': 'yellow' if (feature['properties']['is_val_burst']) else 'blue',
        'weight': 2,
        'fillOpacity': 0.4 * float(feature['properties']['count']) / 100
    }

m = leafmap.Map()
m.add_gdf(df_burst_subset[~df_burst_subset.track_number.isin(TRACKS_TO_EXCLUDE)].copy(), 
          layer_name=f"Burst Data", info_mode='on_click', 
          style_callback=f)
m.add_gdf(df_val_sites_subset, 
      layer_name=f"Val. Sites", info_mode='on_click')
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

# Visualize Burst Time Series (Side by Side)

Source: https://notebooks.gishub.org/leafmap/21_ts_inspector/

In [8]:
BURST_ID = 'T167-357346-IW1'

In [9]:
df_burst = df_burst_subset[df_burst_subset.jpl_burst_id == BURST_ID].reset_index(drop=True)
df_burst

,geometry,jpl_burst_id,orbit_pass,count,is_val_burst,track_number
0,"POLYGON ((37.32737 47.75606, 36.19065 47.90545...",T167-357346-IW1,DESCENDING,102,True,167


In [10]:
df_ts = df_rtc[df_rtc.jpl_burst_id == BURST_ID].reset_index(drop=True)
df_ts.head()

,rtc_s1_id,input_slc_id,jpl_burst_id,bursts_per_slc_input,rtc_s1_vv_url,rtc_s1_vh_url,rtc_s1_h5_url,acq_datetime
0,OPERA_L2_RTC-S1_T167-357346-IW1_20201003T03473...,S1B_IW_SLC__1SDV_20201003T034719_20201003T0347...,T167-357346-IW1,32,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-03 03:47:19
1,OPERA_L2_RTC-S1_T167-357346-IW1_20201009T03481...,S1A_IW_SLC__1SDV_20201009T034811_20201009T0348...,T167-357346-IW1,28,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-09 03:48:11
2,OPERA_L2_RTC-S1_T167-357346-IW1_20201015T03473...,S1B_IW_SLC__1SDV_20201015T034719_20201015T0347...,T167-357346-IW1,32,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-15 03:47:19
3,OPERA_L2_RTC-S1_T167-357346-IW1_20201021T03481...,S1A_IW_SLC__1SDV_20201021T034751_20201021T0348...,T167-357346-IW1,30,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-21 03:47:51
4,OPERA_L2_RTC-S1_T167-357346-IW1_20201027T03473...,S1B_IW_SLC__1SDV_20201027T034719_20201027T0347...,T167-357346-IW1,32,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,https://opera-pst-rs-pop1.s3.us-west-2.amazona...,2020-10-27 03:47:19


In [11]:
layers = {d: leafmap.common.get_local_tile_layer(url, vmin=0, vmax=.25) for (d, url) in zip(tqdm(df_ts.acq_datetime[:]), df_ts.rtc_s1_vh_url)}

100%|███████████████████████████████| 102/102 [00:47<00:00,  2.15it/s]
Exception possibly due to cache backend.
Traceback (most recent call last):
  File "/Users/cmarshak/miniforge3/envs/dist-s1-intel/lib/python3.12/site-packages/flask_caching/__init__.py", line 408, in decorated_function
    self.cache.set(
  File "/Users/cmarshak/miniforge3/envs/dist-s1-intel/lib/python3.12/site-packages/cachelib/simple.py", line 79, in set
    self._prune()
  File "/Users/cmarshak/miniforge3/envs/dist-s1-intel/lib/python3.12/site-packages/cachelib/simple.py", line 56, in _prune
    self._remove_expired(now)
  File "/Users/cmarshak/miniforge3/envs/dist-s1-intel/lib/python3.12/site-packages/cachelib/simple.py", line 37, in _remove_expired
    toremove = [k for k, (expires, _) in self._cache.items() if expires < now]
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: dictionary keys changed during iteration
Exception possibly due to cache backend.
Traceback (mo

In [12]:
centroid = df_burst.geometry[0].centroid
m = leafmap.ts_inspector(layers, center=(centroid.y, centroid.x), zoom=11)
m.add_gdf(df_val_sites, 
      layer_name=f"Val. Sites", info_mode='on_click')
m

Map(center=[47.73190107276903, 36.7266654878584], controls=(AttributionControl(options=['position', 'prefix'],…

# Time Slider

Not very performant and therefore will be omitted.

In [13]:
# m = leafmap.Map(center=(centroid.y, centroid.x), zoom=10)
# m.add_time_slider(layers,
#                   labels=list(layers.keys()),
#                   time_interval=2, 
#                   zoom=11)
# m

# Visualize with ESRI World Imagery

1. https://leafmap.org/notebooks/02_using_basemaps/
2. https://leafmap.org/notebooks/05_load_raster/

In [14]:
import rasterio
from localtileserver import TileClient, get_leaflet_tile_layer

J = 0
src_1 = rasterio.open(df_ts.rtc_s1_vh_url.tolist()[J])
client_1 = TileClient(src_1)

K = -1
src_2 = rasterio.open(df_ts.rtc_s1_vh_url.tolist()[K])
client_2 = TileClient(src_2)

In [15]:
m = leafmap.Map(center=(centroid.y, centroid.x))
m.add_basemap("Esri.WorldImagery")

m.add_raster(client_1, vmin=0, vmax=.25, layer_name=df_ts.acq_datetime.tolist()[J])
K = -1
m.add_raster(client_2, vmin=0, vmax=.25, layer_name=df_ts.acq_datetime.tolist()[K])
m

Map(center=[47.73190107276903, 36.7266654878584], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

# Despeckling

Uses the simplest "homomorphic" despeckling from here: https://www.charles-deledalle.fr/pages/mulog.php

In [16]:
import bm3d
import numpy as np

def despeckle_one(X: np.ndarray, reg_param=.2) -> np.ndarray:
    X_db = np.log10(X, out=np.full(X.shape, np.nan), where=(~np.isnan(X)))
    X_db[np.isnan(X)] = -30
    X_db_dspkl = bm3d.bm3d(X_db, reg_param)
    X_dspkl = np.power(10, X_db_dspkl)
    X_dspkl[np.isnan(X)] = np.nan
    return X_dspkl

In [17]:
N = 0
with rasterio.open(df_ts.rtc_s1_vh_url.tolist()[N]) as ds:
    X_vh = ds.read(1)
    profile = ds.profile

In [18]:
%%time

X_dspkl = despeckle_one(X_vh)

CPU times: user 4min 42s, sys: 38.8 s, total: 5min 21s
Wall time: 38.5 s


## Visualization: Layer Rasters

As above

In [19]:
# Create rasterio dataset in memory
memory_file = rasterio.MemoryFile()
raster_dataset = memory_file.open(driver='GTiff',
                                height=X_dspkl.shape[0],
                                width=X_dspkl.shape[1],
                                count=1,
                                dtype=str(X_dspkl.dtype),
                                crs=profile['crs'],
                                transform=profile['transform'],
                                 nodata=np.nan)

# Write data array values to the rasterio dataset
raster_dataset.write(X_dspkl, 1)
raster_dataset.close()

client_dspkl = TileClient(raster_dataset)

In [20]:
src_orig = rasterio.open(df_ts.rtc_s1_vh_url.tolist()[N])
client_orig = TileClient(src_orig)

In [23]:
m = leafmap.Map(center=(centroid.y, centroid.x), zoom=10)
m.add_basemap("Esri.WorldImagery")

m.add_raster(client_orig, vmin=0, vmax=.25, layer_name='Original')
K = -1
m.add_raster(client_dspkl, vmin=0, vmax=.25, layer_name='Despeckled')
m

Map(center=[47.73190107276903, 36.7266654878584], controls=(ZoomControl(options=['position', 'zoom_in_text', '…

## Split Pane

Reference: https://localtileserver.banesullivan.com/user-guide/compare.html

In [22]:
from ipyleaflet import Map, ScaleControl, FullScreenControl, SplitMapControl

m = leafmap.Map(center=client_dspkl.center(), zoom=12)

# Shared display parameters
display = dict(vmin=0, vmax=.25, colormap='binary_r')

# Create 2 tile layers from different raster
l = get_leaflet_tile_layer(client_dspkl, **display)
r = get_leaflet_tile_layer(client_orig, **display)


control = SplitMapControl(left_layer=l, right_layer=r)
m.add_control(control)
m.add_control(ScaleControl(position='bottomleft'))
m.add_control(FullScreenControl())
m

Map(center=[47.733090000000004, 36.733524], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…